In [1]:
import os,glob
import numpy as np
import pandas as pd
import glob
import pyslha
import time
import progressbar as P
from atlas_susy_2016_08_Recast import getLLPs, getJets,eventAcc,vertexAcc
from ATLAS_data.effFunctions import vertexEff, eventEff
import matplotlib.pyplot as plt
import os,glob
import numpy as np
import seaborn as sns
from scipy.special import kn,zetac
from helper import LLP
import pyslha
sns.set() #Set style
sns.set_style('ticks',{'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
sns.set_palette(sns.color_palette("Paired"))

delphesDir = os.path.abspath("../DelphesLLP")
os.environ['ROOT_INCLUDE_PATH'] = os.path.join(delphesDir,"external")

import ROOT

ROOT.gSystem.Load(os.path.join(delphesDir,"libDelphes.so"))

ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')


Welcome to JupyROOT 6.28/06


True

In [2]:
inputFile = '../pp2BB_atlas_scan_high/Events/run_08/cdfo_530_500.0_tau_5e-02_delphes_events.root'

In [3]:
f = ROOT.TFile(inputFile,'read')
tree = f.Get("Delphes")
nevts = tree.GetEntries()
for ievt in range(nevts):    
    tree.GetEntry(ievt)   
    jets = getJets(tree.GenJet,pTmin=25.,etaMax=5.0)
    met = tree.GenMissingET.At(0).MET
    if met < 200.0:
        continue

    # Event acceptance
    evt_acc = eventAcc(jets,met,metCut=200.0,
                       maxJetChargedPT=5.0,minJetPt1=70.,
                       minJetPt2=25.,minPVdistance=4.0)
    if not evt_acc:
        print(ievt)
        break

8


In [4]:
f = ROOT.TFile(inputFile,'read')
tree = f.Get("Delphes")
nevts = tree.GetEntries()

ievt = 8

tree.GetEntry(ievt)
jets = getJets(tree.GenJet,pTmin=25.,etaMax=5.0)
print(len(jets))
met = tree.GenMissingET.At(0).MET
llps = getLLPs(tree.bsm,tree.bsmDirectDaughters,tree.bsmFinalDaughters)
print(len(llps))
print(met)

5
2
557.328857421875


In [5]:
jets_sorted = sorted(jets, key = lambda j: j.PT, reverse=True)

In [6]:
good_jets = []
for ijet,jet in enumerate(jets_sorted):
    pTCharged = 0.0
    for ip in range(jet.Constituents.GetEntries()):
        particle = jet.Constituents.At(ip)
        if particle.Charge == 0:
            continue
        if particle.Status != 1:
            continue
        r_prod = np.sqrt(particle.X**2 + particle.Y**2)
        if r_prod  > 4.0:
            continue
        pTCharged += particle.PT
    print(ijet,jet.PT,pTCharged)
    if pTCharged > 5.0:
        continue
    good_jets.append(jet)

0 624.55029296875 381.4975438117981
1 84.8246841430664 11.298833914101124
2 50.8814811706543 27.294546976685524
3 46.04990005493164 21.8657443523407
4 38.95610809326172 0.46478143334388733


In [7]:
for j in good_jets:
    print(j.PT,j.Eta,j.Phi)

38.95610809326172 -2.1852800846099854 0.3933102488517761


In [8]:
for llp in llps:
    print(llp.PID)
    for d in llp.directDaughters:
        print('  ',d.PID,d.PT,d.Eta,d.Phi,d.Status)

-1000005
   -5 58.596187591552734 -0.2951020896434784 -2.6238155364990234 23
   1000022 565.7534790039062 -0.49909040331840515 -2.125361204147339 1
1000005
   5 38.54849624633789 -2.174403667449951 0.39536750316619873 23
   1000022 200.81687927246094 -2.974487066268921 -0.33365586400032043 1


In [9]:
import fastjet

In [10]:
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.4)
jetArray = []
pList = []
for iptc in range(tree.StableParticle.GetEntries()):
    p = tree.StableParticle.At(iptc)
    if int(abs(p.PID)) in [12,14,16]:
        continue
    pList.append({"px": float(p.Px), "py": float(p.Py), "pz": float(p.Pz), "E": float(p.E)})
    jetArray.append(fastjet.PseudoJet(float(p.Px),float(p.Py),float(p.Pz),float(p.E)))
print(len(jetArray))

441


In [11]:
jetArrayB = [
    fastjet.PseudoJet(-0.693452, 0.444349, 8.515428, 18.556302),
    fastjet.PseudoJet(-0.292171, 0.719713, 4.673899, 14.740059),
    fastjet.PseudoJet(-0.422724, 0.446733, -4.174223, 14.221597),
    fastjet.PseudoJet(-0.536736, -1.226474, -6.468420, 16.623934)
]

In [12]:
import awkward as ak
jetArrayC = ak.Array(pList[:3])

In [13]:
# cluster = fastjet.ClusterSequence(jetArray[:5], jetdef)
# cluster = fastjet.ClusterSequence(jetArrayB, jetdef)
cluster = fastjet.ClusterSequence(jetArrayC, jetdef)

ValueError: vector::reserve

#--------------------------------------------------------------------------
#                         FastJet release 3.3.4
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code
# and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------


In [ ]:
fast_jets = cluster.inclusive_jets(ptmin = 20.0)

In [ ]:
print(len(fast_jets))
for j in fast_jets:
    print(j.PT)

In [ ]:
fastjet.__version__